In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict, namedtuple, OrderedDict

In [10]:
min_sup = 2
min_conf = 0.1

In [8]:
def load(movies):
  movies = json.loads(movies)
  movies = set(movies)
  return movies
  
inp = pd.read_csv('transaction_mock.csv')
inp['tr'] = inp['tr'].apply(load)
inp

,Id,tr
0,1,"{1, 2, 5}"
1,2,"{2, 4}"
2,3,"{2, 3}"
3,4,"{1, 2, 4}"
4,5,"{1, 3}"
5,6,"{2, 3}"
6,7,"{1, 3}"
7,8,"{1, 2, 3, 5}"
8,9,"{1, 2, 3}"


In [19]:
it_set_1 = set()

for index, row in inp.iterrows():
  Id, tr = row
  it_set_1 = it_set_1.union(tr)

it_set_1 = list(it_set_1)

print('distinct movies:', len(it_set_1))

mp = {}

for index, row in inp.iterrows():
  Id, tr = row
  for mId in tr:
    if mId in mp:
      mp[mId]+=1
    else:
      mp[mId]=1
      
it_df_1 = pd.DataFrame(list(mp.items()), columns=['mId', 'Support'])
it_df_1 = it_df_1.sort_values(by=['mId','Support'],ascending=True)
it_df_1 = it_df_1.sort_values(by=['Support'],ascending=False)
it_df_1 = it_df_1[it_df_1['Support'] >= min_sup]

print('distinct movies after filtering minsup:', it_df_1.shape[0])

it_set_1 = it_df_1['mId'].tolist()

it_df_1

distinct movies: 5
distinct movies after filtering minsup: 5


,mId,Support
1,2,7
0,1,6
4,3,6
3,4,2
2,5,2


In [22]:
def filter_and_sort_by_freq(movies):
    movies = list(movies)
    filtered_movies = filter(lambda x: (x in it_set_1), movies)
    sorted_movies = sorted(filtered_movies, key=lambda x: it_set_1.index(x))
    return sorted_movies

inp['tr'] = inp['tr'].apply(filter_and_sort_by_freq)
inp

,Id,tr
0,1,"[2, 1, 5]"
1,2,"[2, 4]"
2,3,"[2, 3]"
3,4,"[2, 1, 4]"
4,5,"[1, 3]"
5,6,"[2, 3]"
6,7,"[1, 3]"
7,8,"[2, 1, 3, 5]"
8,9,"[2, 1, 3]"


In [74]:
class FPNode:
    def __init__(self, item, count, parent) -> None:
        self.item = item
        self.parent = parent
        self.children = {}
        self.link = None
        self.count = count

    def increment(self, count):
        self.count = self.count + count

    def pretty_print(self):
        print(f'({self.item}-{self.count})[ ', end='')
        for id,ch in self.children.items():
            ch.pretty_print()
        print(' ], ',end='')

class FPTreeBase:
    def __init__(self, transactions, min_sup) -> None:
        self.transactions = transactions
        self.min_sup = min_sup
        self.header_table = defaultdict(list)
        self.root = None

    def _build_tree(self):
        root = FPNode(None, 1, None)
        for transaction in self.transactions:
            self._insert_transaction(root, transaction)
        self.root = root
        return root

    def _insert_transaction(self, node, transaction):
        if len(transaction) == 0:
            return

        [first_item, freq] = transaction[0]
        if first_item in node.children:
            node.children[first_item].increment(freq)
        else:
            new_node = FPNode(first_item, freq, node)
            node.children[first_item] = new_node
            self.header_table[first_item].append(new_node)

        # Recursively insert the rest of the transaction
        remaining_tr = transaction[1:]
        self._insert_transaction(node.children[first_item], remaining_tr)

    def _prefix_path(self, node):
        path = []
        while node and node.item is not None:
            path.append(node.item)
            node = node.parent
        return path[::-1]
    
    def mine_patterns(self):
        patterns = {}
        final_cond_base = []
        # Process items in reverse order of their frequency (from the header table)
        for item in reversed(it_set_1):
            if(item not in self.header_table):
                continue
            
            conditional_patterns = []
            for node in self.header_table[item]:
                path = self._prefix_path(node)
                # print('Path for ', item, ':', path[1:10],'...')
                if path:
                    conditional_patterns.append([[p, node.count] for p in path])

            print(f'Conditional Base Patterns I{item}:', conditional_patterns)
            leaf_item_freq = OrderedDict()
            for pattern in conditional_patterns:
                for [node, freq] in pattern:
                    if node not in leaf_item_freq:
                        leaf_item_freq[node] = freq
                    else:
                        leaf_item_freq[node] += freq
            
            print(leaf_item_freq)
            leaf_item_freq = {k:v for k,v in leaf_item_freq.items() if v >= self.min_sup}
            print(leaf_item_freq)
            
            for pattern in conditional_patterns:
              temp = []
              for [node, freq] in pattern:
                if(node in leaf_item_freq):
                  temp.append([node, freq])
              
              final_cond_base.append(temp)
            
            print(f'Final Conditional Base Patterns I{item}:', conditional_patterns)
            if(len(conditional_patterns)==1):
              if(conditional_patterns[0][0][1] >= self.min_sup):
                patterns[tuple(sorted([node for node, freq in conditional_patterns[0]]))] = conditional_patterns[0][0][1]
            elif(len(final_cond_base) >= 2):
              print('Recursing >>>>>>>>')
              print([p[0:len(p)-1] for p in conditional_patterns])
              conditional_tree = FPTreeBase([p[0:len(p)-1] for p in conditional_patterns], self.min_sup)
              conditional_tree._build_tree()
              
              conditional_tree.root.pretty_print()
              print()
              
              conditional_patterns_freq = conditional_tree.mine_patterns()
              for pattern, count in conditional_patterns_freq.items():
                  patterns[tuple(sorted(list(pattern) + [item]))] = count
              print('BackTracking <<<<<<<<')
              print(f'Patterns ending in I{item}:', conditional_patterns_freq)
              
            item_count = sum([pattern[0][1] for pattern in conditional_patterns])
            if item_count >= self.min_sup:
                patterns[(item,)] = item_count

        # unique_cond_base_set = set(map(tuple,final_cond_base))
        # unique_cond_base_list =list(unique_cond_base_set)

        return patterns
    
class FPTree(FPTreeBase):
    def __init__(self, transactions, min_sup) -> None:
        new_transactions = []
        for transaction in transactions:
            new_transactions.append([[node, 1] for node in transaction])
        super().__init__(new_transactions, min_sup)

    def fit(self):
        self.root = self._build_tree()

In [75]:
fpTree = FPTree(inp['tr'].tolist(), min_sup)
fpTree.fit()
print('FItting done')
patterns = fpTree.mine_patterns()
print(patterns)
# fpTree.visualize_tree(max_depth=2)

FItting done
Conditional Base Patterns I5: [[[2, 1], [1, 1], [5, 1]], [[2, 1], [1, 1], [3, 1], [5, 1]]]
OrderedDict({2: 2, 1: 2, 5: 2, 3: 1})
{2: 2, 1: 2, 5: 2}
Final Conditional Base Patterns I5: [[[2, 1], [1, 1], [5, 1]], [[2, 1], [1, 1], [3, 1], [5, 1]]]
Recursing >>>>>>>>
[[[2, 1], [1, 1]], [[2, 1], [1, 1], [3, 1]]]
(None-1)[ (2-2)[ (1-2)[ (3-1)[  ],  ],  ],  ], 
Conditional Base Patterns I3: [[[2, 1], [1, 1], [3, 1]]]
OrderedDict({2: 1, 1: 1, 3: 1})
{}
Final Conditional Base Patterns I3: [[[2, 1], [1, 1], [3, 1]]]
Conditional Base Patterns I1: [[[2, 2], [1, 2]]]
OrderedDict({2: 2, 1: 2})
{2: 2, 1: 2}
Final Conditional Base Patterns I1: [[[2, 2], [1, 2]]]
Conditional Base Patterns I2: [[[2, 2]]]
OrderedDict({2: 2})
{2: 2}
Final Conditional Base Patterns I2: [[[2, 2]]]
BackTracking <<<<<<<<
Patterns ending in I5: {(1, 2): 2, (1,): 2, (2,): 2}
Conditional Base Patterns I4: [[[2, 1], [4, 1]], [[2, 1], [1, 1], [4, 1]]]
OrderedDict({2: 2, 4: 2, 1: 1})
{2: 2, 4: 2}
Final Conditional Base

In [29]:
print(fpTree.root.item, fpTree.root.children)
print(fpTree.root.children[2].item, fpTree.root.children[2].children)

None {2: <__main__.FPNode object at 0x748535509730>, 1: <__main__.FPNode object at 0x748535509b20>}
2 {1: <__main__.FPNode object at 0x7485355091f0>, 4: <__main__.FPNode object at 0x74853550bec0>, 3: <__main__.FPNode object at 0x74853550b170>}
